In [1]:
from process_bigraph import Process, pp, Step
from typing import *


class ComparisonDocument:
    """To be called 'behind-the-scenes' by the Comparison REST API"""
    def __init__(self, 
                 simulators: List[str], 
                 duration: int, 
                 num_steps: int,
                 model_filepath: str,
                 framework_type='deterministic'):
        self.simulators = simulators
        self.composite = {
            'processes': {
                'duration': duration, 
                'num_steps': num_steps,
            }
        }
        self.framework_type = framework_type
        context = 'concentrations' if 'deterministic' in self.framework_type else 'counts'
        self.species_port_name = f'floating_species_{context}'
        self.species_store = [f'floating_species_{context}_store']
        self._populate_composition(model_filepath)
        self._add_emitter()
    
    def _add_emitter(self):
        self.composite['emitter'] = {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    self.species_port_name: 'tree[float]',
                    'time': 'float'
                }
            },
            'inputs': {
                self.species_port_name: self.species_store,
                'time': ['time_store']
            }
        }
        
    def _populate_composition(self, model_filepath: str):
        context = 'concentrations' if 'deterministic' in self.framework_type else 'counts'
        for process in self.simulators:
            self._add_ode_process_schema(process, context, model={'model_source': model_filepath})
    
    def _add_ode_process_schema(
            self,
            process_name: str, 
            species_context: str,
            **config
            ) -> None:
        species_port_name = f'floating_species_{species_context}'
        species_store = [f'floating_species_{species_context}_store']
        self.composite['processes'][process_name] = {
                '_type': 'process',
                'address': f'local:{process_name}',
                'config': config, 
                'inputs': {
                    species_port_name: species_store,
                    'model_parameters': ['model_parameters_store'],
                    'time': ['time_store'],
                    'reactions': ['reactions_store']
                },
                'outputs': {
                        species_port_name: species_store,
                        'time': ['time_store']
                }
        } 
        
    
    

In [4]:
simulators = ['copasi', 'tellurium']  # , 'amici']
duration = 30
n_steps = 200
model_filepath = '../biosimulator_processes/model_files/sbml/Caravagna2010.xml'


comparison = ComparisonDocument(simulators, duration, n_steps, model_filepath)

In [5]:
pp(comparison.composite)

{ 'emitter': { '_type': 'step',
               'address': 'local:ram-emitter',
               'config': { 'emit': { 'floating_species_concentrations': 'tree[float]',
                                     'time': 'float'}},
               'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                           'time': ['time_store']}},
  'processes': { 'copasi': { '_type': 'process',
                             'address': 'local:copasi',
                             'config': { 'model': { 'model_source': '../biosimulator_processes/model_files/sbml/Caravagna2010.xml'}},
                             'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                                         'model_parameters': [ 'model_parameters_store'],
                                         'reactions': ['reactions_store'],
                                         'time': ['time_store']},
                             

In [ ]:
from process_bigraph import Process, Step, pp




class SimulatorComparison(Process):
    config_schema = {
        'duration': 'number',  
        'simulators': 'list[string]',
        'framework_type': 'maybe[string]'  # ie: 'ode', 'stochastic', "method" types
    }
    
    def __init__(self, config=None, core=None):
        """
        1. create references to simulator processes
        
        """
        super().__init__(config, core)
        
        
 
 
 
 class GeneticParameterOptimizer(Process):
     config_schema = {
         'target_parameter': {  # experimental data
             'name': 'string',
             'target_value': 'float'
         },
         'comparison_results': 'tree[float]'  # sim1Name: values over time, sim2Name: etc
     }
     
     def __init__(self, config=None, core=None):
         super().__init__(config, core)
         
     def calculate_fitness(self, member: float) -> float:
         return self.config['target_parameter']['target_value'] - member